# Advent of Code 2023
## Day 22
*<https://adventofcode.com/2023/day/22>*

In [1]:
import heapq
import math
import re
import functools as ft
from collections import Counter, defaultdict, deque, namedtuple
from itertools import combinations, permutations, product
from string import ascii_letters, ascii_lowercase, ascii_uppercase

import IPython
import z3
from rich import inspect, pretty, print

from new_helper import *

pretty.install()

In [2]:
DAY = 22
input_str = get_aoc_input(DAY, 2023)
part_1 = part_2 = 0

In [3]:
inp = input_str.parse_lines()

In [4]:
from typing import NamedTuple


class Brick(NamedTuple):
    x: range
    y: range
    z: range


bricks: list[Brick] = []

for line in inp:
    x1, y1, z1, x2, y2, z2 = ints(line)
    bricks.append(Brick(range(x1, x2 + 1), range(y1, y2 + 1), range(z1, z2 + 1)))

The key observation is that if we sort the bricks by the lowest points, then after dropping the bricks we have the guarantee that if `i < j`, then `bricks[j]` is not (even by extension) a support of `bricks[i]`, since at some point `bricks[j]` was positioned above (or on the same level as) `bricks[i]`.

In [5]:
bricks.sort(key=lambda b: b.z.start)

In [6]:
def shift_down(b, n: int):
    return Brick(b.x, b.y, range(b.z.start - n, b.z.stop - n))

In [7]:
def overlaps(b1: Brick, b2: Brick) -> bool:
    return (
        max(b1.x.start, b2.x.start) < min(b1.x.stop, b2.x.stop)
        and max(b1.y.start, b2.y.start) < min(b1.y.stop, b2.y.stop)
        and max(b1.z.start, b2.z.start) < min(b1.z.stop, b2.z.stop)
    )

In [8]:
def get_shift(bricks: list[Brick], i: int, supports_of: dict[int, set[int]]) -> int:
    b = bricks[i]
    max_disallowed = 1

    for j, b2 in enumerate(bricks[:i]):
        if b2.z.stop > b.z.start:
            continue

        if b2.z.stop < max_disallowed:
            continue

        shift_by = b.z.start - b2.z.stop + 1
        assert shift_by > 0

        if overlaps(b2, shift_down(b, shift_by)):
            if b2.z.stop == max_disallowed:
                supports_of[i].add(j)
            else:
                supports_of[i] = {j}
                max_disallowed = b2.z.stop

    return b.z.start - max_disallowed

In [9]:
supports_of: dict[int, set[int]] = defaultdict(set[int])
for i, b in enumerate(bricks):
    bricks[i] = shift_down(b, get_shift(bricks, i, supports_of))

is_supporting: dict[int, set[int]] = defaultdict(set[int])
for k, v in supports_of.items():
    for i in v:
        is_supporting[i].add(k)

In [10]:
for i in range(len(bricks)):
    for j in is_supporting[i]:
        if supports_of[j] == {i}:
            break
    else:
        part_1 += 1

In [11]:
for i in range(len(bricks)):
    dropped = set()
    Q = deque([i])

    while Q:
        j = Q.popleft()
        if j in dropped:
            continue

        dropped.add(j)

        for k in is_supporting[j]:
            if len(supports_of[k] - dropped) == 0:
                Q.append(k)

    part_2 += len(dropped) - 1

In [12]:
print_part_1(part_1)
print_part_2(part_2)

Part 1: 454

Part 2: 74287